# Data Loading 

In [ ]:
!pip install cmapy


In [ ]:
import os
import random
import numpy as np
import cv2
import pandas as pd
import torch
import librosa
from tqdm import tqdm
from matplotlib import pyplot as plt
import librosa.display
import cmapy
import io
import math
import os
import torch
import torchvision
import torchvision.transforms as transforms
from torchvision.utils import make_grid
from tqdm.notebook import tqdm as tqdm
import matplotlib.pyplot as plt
import numpy as np
from torchvision import datasets
import torchvision.transforms.functional as F
from torchvision import utils
from torch.utils.data import random_split
from torch.utils.data import TensorDataset
from torch.utils.data import Dataset, DataLoader
import librosa
from tqdm import tqdm
import librosa.display
from utils import *






# Helper Functions

In [ ]:
import os
import random
import numpy as np
import cv2
import pandas as pd
import torch
import librosa
from tqdm import tqdm
from matplotlib import pyplot as plt
import librosa.display
import cmapy
import io
import math
import os
import torch
import torchvision
import torchvision.transforms as transforms
from torchvision.utils import make_grid
from tqdm.notebook import tqdm as tqdm
import matplotlib.pyplot as plt
import numpy as np
from torchvision import datasets
import torchvision.transforms.functional as F
from torchvision import utils
from torch.utils.data import random_split
import torchaudio

def get_annotation_data(file_name,data_dir): # Function returns the recording_data and the annotation_data
  '''
  Parameters
  ----------
  file_name: the file name of which data is to be retrived
  data_dir: Directory where file is present
  '''
  annotation_data = pd.read_csv(os.path.join(data_dir,file_name+".txt"),sep="\t")
  annotation_data.columns = ["start","end","crackle","wheeze"]
  file_data = file_name.split("_")
  recording_data = pd.DataFrame([file_data],columns = ["pid","recording_index","chest_location","acquisition_mode","equipment"])
  return recording_data,annotation_data 


def save_images(image, train_flag):
    save_dir = 'dump_image'
    if not os.path.isdir(save_dir):
        os.makedirs(save_dir)
        
    if train_flag:
        save_dir = os.path.join(save_dir, 'train')
        if not os.path.isdir(save_dir):
            os.makedirs(save_dir)
        cv2.imwrite(os.path.join(save_dir, image[1]+'_'+str(image[2])+'_'+str(image[3])+'_'+str(image[4])+'.jpg'), cv2.cvtColor(image[0], cv2.COLOR_RGB2BGR))
    else:
        save_dir = os.path.join(save_dir, 'test')
        if not os.path.isdir(save_dir):
            os.makedirs(save_dir)
        cv2.imwrite(os.path.join(save_dir, image[1]+'_'+str(image[2])+'_'+str(image[3])+'_'+str(image[4])+'.jpg'), cv2.cvtColor(image[0], cv2.COLOR_RGB2BGR))

def get_label(crackle, wheeze):
    if crackle == 0 and wheeze == 0:
        return 0
    elif crackle == 1 and wheeze == 0:
        return 1
    elif crackle == 0 and wheeze == 1:
        return 2
    else:
        return 3

def get_samples(file_name,dataset_dir,annotation_data,sample_rate):
  samples = []
  audio_file,original_sr = torchaudio.load(os.path.join(dataset_dir,file_name+".wav"))
  audio_file = torchaudio.transforms.Resample(original_sr, sample_rate)(audio_file)
#   print("Audio Chunk shape: ",audio_file.shape)
#   print("Annotation Data: ",annotation_data)
  for i in range(len(annotation_data.index)):
        row = annotation_data.loc[i]
        # print("Row: ",row)
        start = row['start']
        # print("Start: ",start)
        end = row['end']
        crackles = row['crackle']
        wheezes = row['wheeze']
        max_ind = audio_file.shape[1]
        # print("Max Ind",max_ind) 
        # split signal
        start_ind = min(int(start * sample_rate), max_ind)
        end_ind = min(int(end * sample_rate), max_ind)
        # print("Start Ind",start_ind,"End Ind",end_ind)
        audio_chunk = audio_file[:,start_ind:end_ind]
        # print("Audio Chunk",audio_chunk.shape)
        samples.append((audio_chunk, get_label(crackles, wheezes), start,end))
  return samples

def get_train_test_names(train_test_file_names):
    '''
    Get List of file names belonging in train and test datasets
    ---------
    train_test_file_names:  txt file containing names of all samples in train
                            and test as given by ICBHI
    return: train_names,test_names
    '''
    train_test = pd.read_csv(train_test_file_names,sep="\t",header=None)
    train_names = train_test[train_test[1]=="train"]
    test_names = train_test[train_test[1]=="test"]
    return train_names,test_names





# Implementing Custom Dataloader

In [ ]:
from torch.utils.data import Dataset, DataLoader

In [ ]:
from torch.utils.data import Dataset
import librosa
from tqdm import tqdm
import librosa.display
import torchaudio
from utils import *

class ImageLoader(Dataset):
  def __init__(self,dataset_file,dataset_dir,train_flag=True,transform=None):
    self.input_transform = transform
    self.dataset_file = dataset_file
    self.dataset_dir = dataset_dir
    # self.dataset = pd.read_csv(os.path.join(dataset_dir,dataset_file),sep="\t",header=None)
    self.dataset = dataset_file
    self.audio_data = []
    # self.train_flag = False
    # Spectrogram parameters
    self.train_flag = train_flag
    self.sample_rate = 4000
    self.desired_length = 8
    self.n_mels = 128 #128
    self.nfft = 1024 #2048
    self.win_length = int(60/1000*self.sample_rate)
    self.hop = self.win_length//2
    self.f_max = 2000
    self.device_to_files = []  # mapping the filename to device
    # self.patient_to_device = {}
    self.patient_to_samples ={}
    self.patient_to_idx = {}

    # files = os.listdir(dataset_dir)
    failed_files=[]
    print("LOADING AUDIO FILES")
    for i,f in enumerate(tqdm(self.dataset[0])):
      # idx_0: patient_id, idx_1: recording_index, idx_2:Chest location, idx_3:A cquistation mode, idx_4: device
      tokens = f.strip().split("_")
      try:
        _,annotation_data = get_annotation_data(f,dataset_dir)
        sample_data = get_samples(f,dataset_dir,annotation_data,sample_rate=self.sample_rate)
        if tokens[0] not in self.patient_to_samples.keys():
          self.patient_to_samples[tokens[0]] = sample_data
        else:
          self.patient_to_samples[tokens[0]].extend(sample_data)
        
        if tokens[0] not in self.patient_to_idx.keys():
          self.patient_to_idx[tokens[0]] = [i]
        else:
          self.patient_to_idx[tokens[0]].append(i)
        
        
        self.audio_data.extend(sample_data)
        
      except Exception as e:
        print(e)
        failed_files.append(self.dataset.iloc[i,0])
        continue

  
  def augment_audio(self,audio):
    effects = [["lowpass", "-1", "300"],
           ["speed", "0.8"],
           ["rate", f"{self.sample_rate}"],
           ["reverb", "-w"],
           ["channels", "1"],
           ]
    if self.train_flag:
      audio,sr = torchaudio.sox_effects.apply_effects_tensor(audio, self.sample_rate, effects)
    # print(audio.shape)
    return audio
    

    # make a dict for easy indexing and loading

  def __getitem__(self, index):
    if torch.is_tensor(index):
      index = index.tolist()
    audio = self.audio_data[index][0]
    # print(audio.shape)
    # randomly augment data
    # print("Augmenting data")
    audio,_ = torchaudio.sox_effects.apply_effects_tensor(audio,self.sample_rate,effects=[["channels","1"]])
    if np.random.random() > 0.5:
      audio = self.augment_audio(audio)

    # pad the audio to desired length using 
    # print("Before padding: ",audio.shape)
    if audio.shape[1] < self.desired_length*self.sample_rate:
      audio = torch.nn.functional.pad(audio,(0,self.desired_length*self.sample_rate-audio.shape[1]))
    else:
      audio = audio[:,:self.desired_length*self.sample_rate]
    audio_mel_image = torchaudio.transforms.MelSpectrogram(sample_rate=self.sample_rate, n_fft=self.nfft, win_length=self.win_length, hop_length=self.hop, n_mels=self.n_mels, f_max=self.f_max)(audio)
    
    # blank Region Clipping
    audio_mel_image_raw = audio_mel_image.squeeze(0).numpy()
    for row in range(audio_mel_image_raw.shape[0]):
        black_percent = len(np.where(audio_mel_image_raw[row,:]==-100)[0])/len(audio_mel_image_raw[row,:])
        if black_percent > 0.80:
            break
    audio_mel_image_raw = audio_mel_image_raw[:row+1,:]

    for column in range(audio_mel_image_raw.shape[1]):
        black_percent = len(np.where(audio_mel_image_raw[:,column]==-100)[0])/len(audio_mel_image_raw[:,column])
        if black_percent > 0.90:
            break
    
    audio_mel_image_raw = audio_mel_image_raw[:,:column+1]
    audio_mel_image = torch.from_numpy(audio_mel_image_raw).unsqueeze(0)

    
    label = self.audio_data[index][1]
    audio_mel_image = torchvision.transforms.Resize((256,256))(audio_mel_image)
  
    if self.input_transform is not None:
      audio_mel_image = self.input_transform(audio_mel_image)
    
    label = torch.from_numpy(np.array(label)).float()
    return audio_mel_image,label

  def __len__(self):
    # print("Length of dataset: ",len(self.audio_data))
    return len(self.audio_data)
  

# Implementing Patient Based Tuning Model

In [ ]:
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm
from utils import *

class RModel(torch.nn.Module):
    def __init__(self):
        super(RModel,self).__init__()
        self.hidden_dim = 256
        self.num_layers = 1
        self.input_dim = 128
        self.fc_dim = 36864
        self.feature_extractor = torch.nn.Sequential(
            torch.nn.BatchNorm2d(1),
            # conv layer with kernel 5x7
            torch.nn.Conv2d(1, 64, kernel_size=(5,7)),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=(2,3)),
            torch.nn.Conv2d(64,64,kernel_size=(3,3)),
            torch.nn.ReLU(),
            torch.nn.Conv2d(64,64,kernel_size=(3,3)),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=(2,3)),
            torch.nn.Conv2d(64,128,kernel_size=(3,3)),
            torch.nn.ReLU(),
            torch.nn.Conv2d(128,128,kernel_size=(3,3)),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=(2,3)),
            torch.nn.Conv2d(128,128,kernel_size=(3,3)),
            torch.nn.ReLU(),
            torch.nn.Conv2d(128,128,kernel_size=(3,3)),
            torch.nn.ReLU(),
        )
        self.fully_connected = torch.nn.Sequential(
            torch.nn.Flatten(),
            torch.nn.Dropout(0.5),
            torch.nn.Linear(self.fc_dim,100),
            # torch.nn.ReLU(),
            torch.nn.Dropout(0.5),
            torch.nn.Linear(100,4),
            torch.nn.Softmax()
        )
        self.bi_lstm_block = torch.nn.LSTM(self.input_dim, self.hidden_dim, self.num_layers, batch_first=True, bidirectional=True)
    
    
    def init_hidden_state(self, batch_size,device='cuda'):
        return (torch.zeros((self.num_layers * 2, batch_size, self.hidden_dim),device=device),
                torch.zeros((self.num_layers * 2, batch_size, self.hidden_dim),device=device))

    def init_optimizer(self,learning_rate=0.0001):
        return torch.optim.Adam(self.parameters(), lr=learning_rate, amsgrad=False,betas=(0.9, 0.999), weight_decay=5e-4)
    
    def forward(self, x):
        # reshape the input to (batch_size, num_channels, height, width)
        # x = x.permute(0, 3, 2, 1)
        x = self.feature_extractor(x)
        batch_size, C, H, W = x.size()
        x = x.view(batch_size, C, H*W)
        x = x.permute(0, 2, 1)
        self.fc_dim = self.hidden_dim * H * W * 2
        h0, c0 = self.init_hidden_state(batch_size)
        x, (hn, cn) = self.bi_lstm_block(x, (h0, c0))
        x = torch.nn.Tanh()(x)
        x = self.fully_connected(x)
        return x

    
    


def save_model(model,name='model.pt'):
    if not os.path.isdir('saved_models'):
        os.mkdir('saved_models')
      
    torch.save(model.state_dict(), os.path.join('saved_models', name))
    print("Model successfully saved.")
    
def load_model(model):
    model.load_state_dict(torch.load(os.path.join('saved_models', 'model.pt')))
    return model



# Data Loading

In [ ]:

dataset_dir = "ICBHI_final_database"
train_test_txt = "ICBHI_challenge_train_test (1).txt"
BATCH_SIZE = 32





In [ ]:
train_file_df,test_file_df = get_train_test_names(train_test_txt)

In [ ]:
train_dataset = ImageLoader(train_file_df,dataset_dir,train_flag=True)
test_dataset = ImageLoader(test_file_df,dataset_dir,train_flag=False) 
train_dataset,val_dataset = torch.utils.data.random_split(train_dataset,[int(0.8*len(train_dataset)),len(train_dataset)-int(0.8*len(train_dataset))])



In [ ]:
# create a df with audio array, label, start and end
train_class_counts = {}
for i, (audio,label) in enumerate(train_dataset):
    label = int(label)
    if label not in train_class_counts.keys():
        train_class_counts[label] = 1
    else:
        train_class_counts[label]+=1

In [ ]:
train_class_prob = {
    0:train_class_counts[0]/len(train_dataset),
    1:train_class_counts[1]/len(train_dataset),
    2:train_class_counts[2]/len(train_dataset),
    3:train_class_counts[3]/len(train_dataset)
}
train_class_prob

In [ ]:
train_class_weights = torch.from_numpy(np.array([1/train_class_counts[i] for i in range(4)]))

In [ ]:
train_class_weights=train_class_weights.to("cuda").float()
train_class_weights

In [ ]:
train_loader = DataLoader(train_dataset,batch_size=BATCH_SIZE,shuffle=True)
val_loader = DataLoader(val_dataset,batch_size=BATCH_SIZE,shuffle=True)
test_loader = DataLoader(test_dataset,batch_size=BATCH_SIZE,shuffle=True)


# Training


In [ ]:
def get_evaluation_metrics(preds,targets):
    # calculate specificity
    # sepcificity = Correct Label[0] + Correct Label[1] + Correct Label[2] + Correct Label[3] / Total Label[0] + Total Label[1] + Total Label[2] + Total Label[3]
    sensitivity_num = 0
    sensitivity_denm = 0
    for i in range(4):
        sensitivity_num += torch.sum((preds==i) & (targets==i))
        sensitivity_denm += torch.sum(targets==i)
    sensitivity = float(sensitivity_num)/float(sensitivity_denm)
    specificity = float(torch.sum((preds==0) & (targets==0)))/float(torch.sum(targets==0))
    score = 0.5*(sensitivity+specificity)
    return sensitivity,specificity,score

def get_scores(model, testloader, device):
    # model.eval()
    with torch.no_grad():
        val_loss, val_sensitivity, val_specificity, val_score = 0.0, 0.0, 0.0, 0.0
        for val_batch in testloader:
            imgs, targets = val_batch
            imgs, targets = imgs.to(device), targets.to(device).long()
            val_outputs = model(imgs)
            val_loss += torch.nn.CrossEntropyLoss()(val_outputs, targets).item()
            val_preds = torch.argmax(val_outputs, dim=1)
            sensitivity,specificity,score = get_evaluation_metrics(val_preds,targets)
            val_sensitivity += sensitivity
            val_specificity += specificity
            val_score += score

    val_loss = float(float(val_loss)/float(len(testloader)*BATCH_SIZE))
    val_sensitivity = float(float(val_sensitivity)/float(len(testloader)))
    val_specificity = float(float(val_specificity)/float(len(testloader)))
    val_score = float(float(val_score)/float(len(testloader)))
    return val_loss,val_sensitivity,val_specificity,val_score



def evaluate(model, testloader, device):
    # model.eval()
    with torch.no_grad():
        val_loss, val_acc = 0.0, 0.0
        for val_batch in testloader:
            imgs, targets = val_batch
            #print(imgs.shape, targets.shape)
            #print(targets)
            imgs, targets = imgs.to(device), targets.to(device).long()
            val_outputs = model(imgs)
            val_loss += torch.nn.CrossEntropyLoss()(val_outputs, targets).item()
            val_preds = torch.argmax(val_outputs, dim=1)
            val_acc += torch.sum(val_preds == targets)

    val_acc = float(float(val_acc)/float(len(testloader)*BATCH_SIZE))
    return val_acc,val_loss

def train_evaluate(model, optimizer, trainloader, valloader, num_epochs=10, save_model_name=None, dropout=None, learning_rate=0.001, device="cuda"):
    model.to(device)
    model.train()
    train_accuracy_with_epochs = []
    val_accuracy_with_epochs = []
    loss_with_epochs = []
    val_loss_with_epochs = []
    # best_val_acc = 0.0
    best_loss = 0.0345

    for epoch in range(num_epochs):
        train_loss, train_acc = 0, 0
        print("\nEpoch: ", str(epoch+1), "/", str(num_epochs))

        with tqdm(total=len(trainloader)) as pbar:
            for idx, batch in enumerate(trainloader):
                images, labels = batch
                images, labels = images.to(device), labels.to(device).long()
                preds = model(images)
                loss = torch.nn.CrossEntropyLoss(weight=train_class_weights)(preds, labels)
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()
                # print(torch.argmax(preds, dim=1),labels)
                train_loss += loss.item()
                acc = torch.sum(torch.argmax(preds, dim=1) == labels)
                train_acc += acc
                pbar.set_postfix(Loss='{0:.4f}'.format(loss.item()), Accuracy='{0:.4f}'.format(float(train_acc.item()/(BATCH_SIZE*(idx+1)))))
                pbar.update(1)

            val_acc, val_loss = evaluate(model, valloader, device)
            val_accuracy_with_epochs.append(val_acc)
            val_loss_with_epochs.append(val_loss)
            loss_with_epochs.append(train_loss)
            train_loss = float(float(train_loss)/float(len(trainloader)*BATCH_SIZE))
            print("train_acc:", round(float(float(train_acc)/float(len(trainloader)*BATCH_SIZE)), 4), " val_acc:", round(val_acc, 4))
            train_accuracy_with_epochs.append(round(float(float(train_acc)/float(len(trainloader)*BATCH_SIZE)), 4))
            
            if train_loss <= best_loss:
                best_loss = train_loss
                if save_model_name is not None:
                    torch.save(model.state_dict(), save_model_name)
                    print("Model saved at", save_model_name)
                    print("Best train loss:", best_loss)

    return train_accuracy_with_epochs, val_accuracy_with_epochs, model, loss_with_epochs, val_loss_with_epochs


In [ ]:
device="cuda"

# Training Mobile NET 


In [ ]:
mobilenet_model = torchvision.models.mobilenet_v2(weights=torchvision.models.MobileNet_V2_Weights)
new_conv = torch.nn.Conv2d(1, 32, kernel_size=3, stride=2, padding=1, bias=False)
# with torch.no_grad():
#     new_conv.weight[:, 0, :, :] = mobilenet_model.features[0].weight.sum(dim=1)
mobilenet_model.features[0] = new_conv
mobilenet_in_features = mobilenet_model.classifier[1].in_features
mobilenet_model.classifier[1] = torch.nn.Linear(mobilenet_in_features,4)

mobile_optimizer = torch.optim.Adam(mobilenet_model.parameters(),lr=0.0001,betas=(0.9,0.99),amsgrad=False)
mobilenet_train_acc,mobilenet_val_acc,mobilenet_model,mobilenet_loss_with_epochs, mobilenet_val_loss_with_epochs = train_evaluate(mobilenet_model,mobile_optimizer,train_loader,val_loader,num_epochs=50,save_model_name="mobilenetmodel.pt",dropout=0.5,learning_rate=0.0001,device=device)


In [ ]:
import seaborn as sns
sns.set_theme()
plt.plot(mobilenet_val_acc,label="Validation")
plt.plot(mobilenet_train_acc,label="Train")
plt.xlabel("Epochs")
plt.ylabel("Accuracy")
plt.legend()
plt.title("Training vs Validation Accuracy for Mobilenet backbone")
plt.savefig("plots/mobilenet_train_vs_validation_acc.png")
plt.show()

plt.figure()
plt.plot(mobilenet_loss_with_epochs,label="Train")
plt.plot(mobilenet_val_loss_with_epochs,label="Validation")
plt.title("MobileNet-Loss Curve")
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.legend()
plt.savefig("plots/mobilenet_train_vs_val_loss.png")
plt.show()

# Training VGG16 model

In [ ]:
device="cuda"
vgg16_model = torchvision.models.vgg16(weights=torchvision.models.VGG16_Weights.DEFAULT)
vgg16_model.features[0]=torch.nn.Conv2d(1,64,kernel_size=3,padding=1,stride=1)
vgg_in_features = vgg16_model.classifier[6].in_features
vgg16_model.classifier[6] = torch.nn.Linear(vgg_in_features,4)
vgg_optimizer = torch.optim.Adam(vgg16_model.parameters(),lr=0.0001,betas=(0.9,0.99),amsgrad=False)
vgg16_train_acc,vgg16_val_acc,vgg16_model,vgg16_loss_with_epochs,vgg16_val_loss_with_epochs = train_evaluate(vgg16_model,vgg_optimizer,train_loader,val_loader,num_epochs=50,save_model_name="vgg16model.pt",dropout=0.5,learning_rate=0.0001,device=device)


In [ ]:
plt.figure()
plt.plot(vgg16_val_acc,label="Validation")
plt.plot(vgg16_train_acc,label="Train")
plt.legend()
plt.title("Training vs Validation Accuracy for Vgg16 backbone")
plt.xlabel("Epochs")
plt.ylabel("Accuracy")
plt.savefig("plots/Vgg16_train_vs_validation_acc.png")

plt.figure()
plt.plot(vgg16_loss_with_epochs,label="Train")
plt.plot(vgg16_val_loss_with_epochs,label="Validation")
plt.title("Vgg16-Loss Curve")
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.legend()
plt.savefig("plots/Vgg16_train_vs_val_loss.png")

# Training Hybrid CNN-RNN Model

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
model = RModel()
optimizer = torch.optim.Adam(model.parameters(),lr=0.0001,betas=(0.99,0.99),amsgrad=False)
train_acc,val_acc,model,loss_with_epochs,val_loss_with_epochs = train_evaluate(model,optimizer,train_loader,val_loader,num_epochs=50,save_model_name="model.pt",dropout=0.5,learning_rate=0.0001,device=device)


In [ ]:
import seaborn as sns
sns.set_theme()
plt.figure()
plt.plot(val_acc,label="Validation")
plt.plot(train_acc,label="Train")
plt.legend()
plt.title("Training vs Validation Accuracy for Patient Based Model")
plt.savefig("plots/patient_based_model_train_vs_validation_acc.png")
plt.show()

plt.figure()
plt.plot(loss_with_epochs,label="Train")
plt.plot(val_loss_with_epochs,label="Validation")
plt.title("Patient Based Model-Loss Curve")
plt.legend()
plt.savefig("plots/patient_based_model_train_vs_val_loss.png")
plt.show()

## Fine tune the patient based model for a particular patient

In [ ]:
patient_wise_tune_data = train_dataset.dataset.patient_to_idx
# patient to samples is a dict containing patient id as key and list of samples as value
# we need to fine tune the model patient wise
# we will use the same model as before

patient_id = "107"
patient_samples = patient_wise_tune_data[patient_id]
patient_train_samples = patient_samples[:int(0.8*len(patient_samples))]
patient_test_samples = patient_samples[int(0.8*len(patient_samples)):]
patient_train_dataset = torch.utils.data.Subset(train_dataset,patient_train_samples)
patient_test_dataset = torch.utils.data.Subset(train_dataset,patient_test_samples)
patient_train_loader = torch.utils.data.DataLoader(patient_train_dataset,batch_size=32,shuffle=True)
patient_test_loader = torch.utils.data.DataLoader(patient_test_dataset,batch_size=32,shuffle=True)



In [ ]:
fine_tune_optimizer = torch.optim.Adam(model.parameters(),lr=0.0001,betas=(0.9,0.99),amsgrad=False,weight_decay=0.0001)
fine_tune_train_acc,fine_tune_val_acc,fine_tune_model,fine_tune_loss_with_epochs,fine_tune_val_loss_with_epochs = train_evaluate(model,fine_tune_optimizer,patient_train_loader,patient_test_loader,num_epochs=200,save_model_name="fine_tune_model.pt",dropout=0.5,learning_rate=0.0001,device=device)

# Results

In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
# load the saved models for direct testing
# fine_tune_model = RModel()
# fine_tune_model.load_state_dict(torch.load("saved_models/fine_tune_model.pt"))
# fine_tune_model = fine_tune_model.to(device)


In [ ]:
# load the saved models for direct testing
# vgg16_model = torchvision.models.vgg16(weights=torchvision.models.VGG16_Weights.DEFAULT)
# vgg16_model.features[0]=torch.nn.Conv2d(1,64,kernel_size=3,padding=1,stride=1)
# vgg_in_features = vgg16_model.classifier[6].in_features
# vgg16_model.classifier[6] = torch.nn.Linear(vgg_in_features,4)

# vgg16_model.load_state_dict(torch.load("saved_models/vgg16model.pt"))
# vgg16_model = vgg16_model.to(device)

In [ ]:
# load the saved models for direct testing
# mobilenet_model = torchvision.models.mobilenet_v2(weights=torchvision.models.MobileNet_V2_Weights)
# new_conv = torch.nn.Conv2d(1, 32, kernel_size=3, stride=2, padding=1, bias=False)
# # with torch.no_grad():
# #     new_conv.weight[:, 0, :, :] = mobilenet_model.features[0].weight.sum(dim=1)
# mobilenet_model.features[0] = new_conv
# mobilenet_in_features = mobilenet_model.classifier[1].in_features
# mobilenet_model.classifier[1] = torch.nn.Linear(mobilenet_in_features,4)
# mobilenet_model.load_state_dict(torch.load("saved_models/mobilenetmodel.pt"))
# mobilenet_model = mobilenet_model.to(device)

In [ ]:
print("Results on Testing Data")
loss,specificity, sensitivity, score = get_scores(fine_tune_model, test_loader, device)
print("Test Loss on fine tuned patient specific model: ",loss)
print("Test Specificity on fine tuned patient specific model: ",specificity*100)
print("Test Sensitivity on fine tuned patient specific model: ",sensitivity*100)
print("Test Score on fine tuned patient specific model: ",score*100)


In [ ]:
print("Results on Testing Data")
loss,specificity, sensitivity, score = get_scores(vgg16_model, test_loader, device)
print("Test Loss on fine tuned Vgg16 backbone model: ",loss)
print("Test Specificity on fine tuned Vgg16 backbone model: ",specificity)
print("Test Sensitivity on fine tuned Vgg16 backbone model: ",sensitivity)
print("Test Score on fine tuned Vgg16 backbone model: ",score)


In [ ]:
print("Results on Testing Data")
loss,specificity, sensitivity, score = get_scores(mobilenet_model, test_loader, device)
print("Test Loss on fine tuned Mobilenet backbone model: ",loss)
print("Test Specificity on fine tuned Mobilenet backbone model: ",specificity)
print("Test Sensitivity on fine tuned Mobilenet backbone model: ",sensitivity)
print("Test Score on fine tuned Mobilenet backbone model: ",score)


In [ ]:
print("Results on patient 107 using fine tuned patient specific model")
loss,specificity, sensitivity, score = get_scores(fine_tune_model, patient_test_loader, device)
print("Test Loss for patient 107 on fine tuned patient specific model: ",loss)
print("Test Specificity for patient 107 on fine tuned patient specific model: ",specificity*100)
print("Test Sensitivity for patient 107 on fine tuned patient specific model: ",sensitivity*100)
print("Test Score for patient 107 on fine tuned patient specific model: ",score*100)
print("-------------------------------------------")
print("Results on patient 107 using fine tuned Vgg16 backbone model")
loss,specificity, sensitivity, score = get_scores(vgg16_model, patient_test_loader, device)
print("Test Loss for patient 107 on fine tuned Vgg16 backbone model: ",loss)
print("Test Specificity for patient 107 on fine tuned Vgg16 backbone model: ",specificity*100)
print("Test Sensitivity for patient 107 on fine tuned Vgg16 backbone model: ",sensitivity*100)
print("Test Score for patient 107 on fine tuned Vgg16 backbone model: ",score*100)
print("-------------------------------------------")
print("Results on patient 107 using fine tuned Mobilenet backbone model")
loss,specificity, sensitivity, score = get_scores(mobilenet_model, patient_test_loader, device)
print("Test Loss for patient 107 on fine tuned Mobilenet backbone model: ",loss)
print("Test Specificity for patient 107 on fine tuned Mobilenet backbone model: ",specificity*100)
print("Test Sensitivity for patient 107 on fine tuned Mobilenet backbone model: ",sensitivity*100)
print("Test Score for patient 107 on fine tuned Mobilenet backbone model: ",score*100)
print("-------------------------------------------")


In [ ]:
# print model summary
import torchsummary
print("Patient Based Model Summary")
torchsummary.summary(fine_tune_model,(1,256,256))



In [ ]:
print("MobileNet Model Summary")
torchsummary.summary(mobilenet_model,(1,256,256))


In [ ]:
print("Vgg16 Model Summary")
torchsummary.summary(vgg16_model,(1,256,256))